# Exploring Panel Widgets

Here we test connecting widgets to plots.

In [ ]:
import param
import numpy as np 
import pandas as pd
import panel as pn

import altair as alt
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
from bokeh.models.formatters import DatetimeTickFormatter
from holoviews import opts

hv.extension('bokeh')
pn.extension()

In [ ]:
formatter = DatetimeTickFormatter(months='%b')
formatter2 = DatetimeTickFormatter(months='%b %Y')

In [ ]:
# Styles and plot options used in this user guide
opts.defaults(
    opts.Area(fill_color='cornsilk', line_width=1,
              line_color='black'),
    opts.Ellipse(bgcolor='white', color='black'),
    opts.HLine(color='red', line_dash='dashed'))

In [ ]:
# Do this to plot data binned by month.
#sst.hvplot.violin(by='index.month')

In [ ]:
#%%time
#data_1990 = pd.read_csv('./Boulder_TMAX_1990.csv')


In [ ]:
#data_1990.resample('W', label='left').sum()

In [ ]:
#datasets = {}
#datasets['1990'] = data_1990.pivot_table(index='time', columns='member_id', values='TREFHT')
#datasets['2090'] = data_2090.pivot_table(index='time', columns='member_id', values='TREFHT')
#datasets

In [ ]:
#print(data_1990.info())

In [ ]:
#data_1990.groupby('time')['TREFHTMX'].agg([np.min,np.max])

In [ ]:
#data_1990.head()

In [ ]:
#for label, content in datasets['1990'].items():
#    print(content.to_numpy())

In [ ]:
#def threshold_line(thresh):
#    return hv.HLine(thresh) 
#
#threshold = hv.DynamicMap(threshold_line, kdims='thresh').redim.range(t=(-40,120))



In [ ]:
class CityPlanner(param.Parameterized):
    yearLeft = param.Integer(1990, bounds=(1920, 2100), step=10)
    yearRight = param.Integer(2090, bounds=(1920, 2100), step=10)
    threshold = param.Integer(95, bounds=(0, 150))
    plot_type = param.ObjectSelector(default="mean + spread", objects=["mean + spread", "median + spread", "line", "scatter", "violin", "box + whisker"])
    city = param.ObjectSelector(default="Boulder", objects=["Boulder", "Austin", "Chicago"])
    bin_size = param.ObjectSelector(default="week", objects=["day", "week", "month"])
    units = param.ObjectSelector(default="Fahrenheit", objects=["Fahrenheit", "Celsius", "Kelvins"])
    show_legend = param.Boolean(default=False)
    show_grid = param.Boolean(default=True)

    def get_data(self, isLeft):
        if isLeft:
            year = self.yearLeft
        else:
            year = self.yearRight
        df = pd.read_csv(f'./data/{self.city}_TMAX_{year}.csv')
        df['time'] = pd.to_datetime(df['time'])
        year_delta = 2000 - year
        df['time'] = df['time'].apply(lambda x: x + pd.DateOffset(years=year_delta))
        data = df.pivot_table(index='time', columns='member_id', values='TREFHTMX')
        #print(data.head())
        return data, year
    
    def merge_columns(self, data):
        data = data.reset_index()
        dat2 = pd.melt(data, id_vars='time')
        dat2.index = dat2['time']
        dat2.drop(['time'], axis=1, inplace=True)
        return dat2
    
    def resample(self,data):
        if self.bin_size == 'week':
            data2 = data.copy().resample('W').mean()
            data2['days_above_mean'] = data['days_above_mean'].resample('W').sum()
            data2['days_above_median'] = data['days_above_median'].resample('W').sum()
            ylimit = 8
        elif self.bin_size == 'month':
            data2 = data.copy().resample('M').mean()
            data2['days_above_mean'] = data['days_above_mean'].resample('M').sum()
            data2['days_above_median'] = data['days_above_median'].resample('M').sum()
            ylimit = 32
        else:
            data2 = data
            ylimit = 2
        return data2, ylimit
    
    def temp_convert(self, data):
        if self.units == "Fahrenheit":
            data = (data - 273.0) * (9.0/5.0) + 32.0
        elif self.units == "Celsius":
            data -= 273.0
        else:
            # Do nothing and return data in native units
            pass
        return data
    
    def hvplot_view_left(self):
        plot = self.hvplot_helper(True)
        return plot
    
    def hvplot_view_right(self):
        plot = self.hvplot_helper(False)
        return plot
    
    def hvplot_helper(self, isLeft):
        (data, year)  = self.get_data(isLeft)
        # Convert to the selected units
        data = self.temp_convert(data)
        dat2 = self.merge_columns(data)
        columnIntegers = data.columns[1:].values.tolist()
        columnLabels = [str(x) for x in columnIntegers]
        title = f'{self.city} Max Temperature'
        ylabel = self.units
        xlabel = f'{year}'
        font_bump = '140%'
        bin_label = 'index.' + self.bin_size
        data_mean = data.agg([np.mean], axis=1)
        data_median = data.agg([np.median], axis=1)
        data['days_above_mean'] = (data_mean > self.threshold).astype(int)
        data['days_above_median'] = (data_median > self.threshold).astype(int)
        
        # Compute dat2 before resampling data, for now. 
        data, ylimit = self.resample(data)
                        
        if self.plot_type == "mean + spread":
            data_stats = data[columnIntegers].agg([np.min, np.max, np.mean], axis=1)
            data_stats.index = data.index
            plot = ((data_stats.hvplot.area(x='time',  y='amin', y2='amax', xformatter=formatter, title=title, 
                                            xlabel=xlabel, ylabel=ylabel, legend=self.show_legend, grid=self.show_grid, width=500, 
                                            height=400, alpha=0.4).opts(fontsize={'labels': font_bump}) * data_stats['mean'].hvplot(legend=self.show_legend)  * hv.HLine(self.threshold)) + \
                   (data.hvplot.bar(x='time', y='days_above_mean', xformatter=formatter2, title="Expected Days Above Threshold", xlabel=xlabel, ylabel='Number of Days', width=500, height=400, ylim=(0,ylimit))).opts(fontsize={'labels': font_bump})).cols(1)
        elif self.plot_type == "median + spread":
            data_stats = data[columnIntegers].agg([np.min, np.max, np.median], axis=1)
            data_stats.index = data.index
            plot = ((data_stats.hvplot.area(x='time',  y='amin', y2='amax', xformatter=formatter, title=title, 
                                          xlabel=xlabel, ylabel=ylabel, legend=self.show_legend, grid=self.show_grid, width=500, 
                                          height=400, alpha=0.4).opts(fontsize={'labels': font_bump}) * data_stats['median'].hvplot(legend=self.show_legend) * hv.HLine(self.threshold))  + \
                   (data.hvplot.bar(x='time', y='days_above_median', xformatter=formatter2, title="Expected Days Above Threshold", xlabel=xlabel, ylabel='Number of Days', width=500, height=400, ylim=(0,ylimit))).opts(fontsize={'labels': font_bump})).cols(1)
        elif self.plot_type == "line":
            plot  = ((data.hvplot.line(x='time', y=columnLabels, xformatter=formatter, title=title, 
                                       xlabel=xlabel, ylabel=ylabel, legend=self.show_legend, grid=self.show_grid, width=500, 
                                       height=400).opts(fontsize={'labels': font_bump}) * hv.HLine(self.threshold)) + \
                    (data.hvplot.bar(x='time', y='days_above_mean', xformatter=formatter2, title="Expected Days Above Threshold", xlabel=xlabel, ylabel='Number of Days', width=500, height=400, ylim=(0,ylimit))).opts(fontsize={'labels': font_bump})).cols(1) 
            #print(plot)
        elif self.plot_type == "violin":
            #print(dat2.head())
            plot  = ((dat2.hvplot.violin(by=bin_label, y='value', title=title, 
                                       xlabel=xlabel, ylabel=ylabel, legend=self.show_legend, grid=self.show_grid, width=500, 
                                       height=400).opts(fontsize={'labels': font_bump}) * hv.HLine(self.threshold))  + \
                    (data.hvplot.bar(x='time', y='days_above_mean', xformatter=formatter2, title="Expected Days Above Threshold", xlabel=xlabel, ylabel='Number of Days', width=500, height=400, ylim=(0,ylimit))).opts(fontsize={'labels': font_bump})).cols(1) 
        elif self.plot_type == "box + whisker":
            #print(dat2.head())
            plot  = ((dat2.hvplot.box(by=bin_label, y='value', title=title, 
                                        xlabel=xlabel, ylabel=ylabel, legend=self.show_legend, grid=self.show_grid, width=500, 
                                        height=400).opts(fontsize={'labels': font_bump}) * hv.HLine(self.threshold))  + \
                    (data.hvplot.bar(x='time', y='days_above_mean', xformatter=formatter2, title="Expected Days Above Threshold", xlabel=xlabel, ylabel='Number of Days', width=500, height=400, ylim=(0,ylimit))).opts(fontsize={'labels': font_bump})).cols(1) 
        else:
            plot = ((data.hvplot.scatter(x='time', y=columnLabels, xformatter=formatter, title=title, 
                                        xlabel=xlabel, ylabel=ylabel, legend=self.show_legend, grid=self.show_grid, width=500, 
                                        height=400).opts(fontsize={'labels': font_bump}) * hv.HLine(self.threshold))  + \
                   (data.hvplot.bar(x='time', y='days_above_mean', xformatter=formatter2, title="Expected Days Above Threshold", xlabel=xlabel, ylabel='Number of Days', width=500, height=400, ylim=(0,ylimit))).opts(fontsize={'labels': font_bump})).cols(1) 

        return plot
    
gm = CityPlanner(name='')

In [ ]:
pn.Column(gm.param,
          pn.Row(gm.hvplot_view_left,    gm.hvplot_view_right))

In [ ]:
logo  = """<a href="http://panel.pyviz.org">
           <img src="https://panel.pyviz.org/_static/logo_stacked.png" 
            width=100 height=80 align="left" margin=20px>"""
title = '<h2>City Planner Climate Explorer</h2>'

desc = pn.pane.HTML("""
    The <a href="http://panel.pyviz.org">Panel</a> library from <a href="http://pyviz.org">PyViz</a> 
    lets you make widget-controlled apps and dashboards from a wide variety of 
    plotting libraries and data types. """, width=250)

pn.Row(pn.Column(logo, title, desc, pn.panel(gm.param)),
                 pn.Row(gm.hvplot_view_left,    gm.hvplot_view_right)).servable()